In [7]:
import os
from elasticsearch import Elasticsearch
from typing import List, Dict, Any, Optional
import json

class ProductSearcher:
    def __init__(self, host: str, api_key: str, index_name: str = "products"):
        """
        Initialize Elasticsearch connection
        
        Args:
            host: Elasticsearch host
            index_name: Name of the index containing products
        """
        self.es = Elasticsearch(
            hosts=host,
            api_key=api_key
        )
        self.index_name = index_name
    
    def search_products(self, query_text: str, size: int = 10) -> List[Dict[str, Any]]:
        """
        Search for products using case-insensitive multi-field search
        
        Args:
            query_text: Search term (e.g., "sữa", "sữa vinamilk", "củ dền")
            size: Maximum number of results to return
            
        Returns:
            List of matching products with their scores
        """
        # Build multi-field search query with case insensitive matching
        search_body = {
            "query": {
                "bool": {
                    "should": [
                        # Exact match on product name (highest priority)
                        {
                            "match": {
                                "product_name": {
                                    "query": query_text,
                                    "boost": 3.0,
                                    "fuzziness": "AUTO"
                                }
                            }
                        },
                        # Match in product description
                        {
                            "match": {
                                "product_description": {
                                    "query": query_text,
                                    "boost": 2.0
                                }
                            }
                        },
                        # Match in category
                        {
                            "match": {
                                "category": {
                                    "query": query_text,
                                    "boost": 1.5
                                }
                            }
                        },
                        # Wildcard search for partial matches
                        {
                            "wildcard": {
                                "product_name.keyword": {
                                    "value": f"*{query_text.lower()}*",
                                    "boost": 1.0,
                                    "case_insensitive": True
                                }
                            }
                        }
                    ],
                    "minimum_should_match": 1
                }
            },
            "size": size,
            "sort": [
                "_score",
                {"price": {"order": "asc"}}  # Secondary sort by price
            ]
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_body)
            
            results = []
            for hit in response["hits"]["hits"]:
                product = hit["_source"]
                product["_score"] = hit["_score"]
                product["_id"] = hit["_id"]
                results.append(product)
            
            return results
            
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    def search_by_category(self, category: str, size: int = 10) -> List[Dict[str, Any]]:
        """
        Search products by category (case-insensitive)
        
        Args:
            category: Category to search for
            size: Maximum number of results
            
        Returns:
            List of products in the category
        """
        search_body = {
            "query": {
                "match": {
                    "category": {
                        "query": category,
                        "fuzziness": "AUTO"
                    }
                }
            },
            "size": size,
            "sort": [{"price": {"order": "asc"}}]
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_body)
            
            results = []
            for hit in response["hits"]["hits"]:
                product = hit["_source"]
                product["_id"] = hit["_id"]
                results.append(product)
            
            return results
            
        except Exception as e:
            print(f"Category search error: {e}")
            return []
    
    def suggest_products(self, query_text: str, size: int = 5) -> List[str]:
        """
        Get product name suggestions for autocomplete
        
        Args:
            query_text: Partial product name
            size: Number of suggestions
            
        Returns:
            List of suggested product names
        """
        search_body = {
            "suggest": {
                "product_suggest": {
                    "prefix": query_text.lower(),
                    "completion": {
                        "field": "product_name.suggest",
                        "size": size,
                        "skip_duplicates": True
                    }
                }
            }
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_body)
            suggestions = []
            
            for option in response["suggest"]["product_suggest"][0]["options"]:
                suggestions.append(option["text"])
            
            return suggestions
            
        except Exception as e:
            print(f"Suggestion error: {e}")
            return []

# Usage example
def main():
    # Initialize searcher
    searcher = ProductSearcher(
        host=os.getenv("ELASTICSEARCH_HOST"),
        api_key=os.getenv("ELASTICSEARCH_API_KEY"),
        index_name="product-data"  # Replace with your actual index name
    )
    
    # Example searches
    search_queries = ["sữa", "sữa vinamilk", "củ dền", "RAU", "nước"]
    
    for query in search_queries:
        print(f"\n=== Searching for: '{query}' ===")
        results = searcher.search_products(query, size=5)
        
        if results:
            for i, product in enumerate(results, 1):
                print(f"{i}. {product['product_name']}")
                print(f"   Category: {product['category']}")
                print(f"   Price: {product['price']:,} VND")
                print(f"   Description: {product['product_description']}")
                print(f"   Score: {product['_score']:.2f}")
                print(f"   Discount: {product.get('discount', 0)}%")
                print("-" * 50)
        else:
            print("No products found")
    
    # Category search example
    print(f"\n=== Products in 'RAU - CỦ - QUẢ' category ===")
    category_results = searcher.search_by_category("RAU - CỦ - QUẢ")
    
    for product in category_results:
        print(f"- {product['product_name']} - {product['price']:,} VND")

if __name__ == "__main__":
    main()


=== Searching for: 'sữa' ===
1. Sữa Tươi Tiệt Trùng Nutifood 100% Sữa Tươi Hộp 1L
   Category: BƠ - SỮA - TRỨNG
   Price: 31,000 VND
   Description: Sữa tươi tiệt trùng 100% sữa tươi, giàu dinh dưỡng, thơm ngon tự nhiên.
   Score: 19.45
   Discount: 0%
--------------------------------------------------
2. Sữa Chua Vinamilk Không Đường Túi 100g
   Category: BƠ - SỮA - TRỨNG
   Price: 7,000 VND
   Description: Sữa chua không đường, tiện lợi, dễ sử dụng.
   Score: 16.75
   Discount: 15%
--------------------------------------------------
3. Quả Vú Sữa
   Category: RAU - CỦ - QUẢ
   Price: 40,000 VND
   Description: Quả vú sữa ngọt dịu, thơm ngon.
   Score: 16.66
   Discount: 0%
--------------------------------------------------
4. Sữa Chua Vinamilk Nha Đam Lốc 4 Hộp
   Category: BƠ - SỮA - TRỨNG
   Price: 28,000 VND
   Description: Sữa chua nha đam, thanh mát, giải nhiệt.
   Score: 16.52
   Discount: 15%
--------------------------------------------------
5. Sữa Chua Uống Yakult Lốc 5 Chai